# Transforming between CIE Chromaticity and standard RGB

In [1]:
import numpy as np

![title](./sRGB_in_CIE.png)

## xyY -> sRGB

In [2]:
def XYZ_to_RGBlinear(XYZ):
    M = np.array([[+3.24096994, -1.53738318, -0.49861076],
                  [-0.96924364, +1.87596750, +0.04155506],
                  [+0.05563008, -0.20397696, +1.05697151]])
    return M.dot(XYZ)
    
def gamma(u):
    if isinstance(u, np.ndarray):
        return np.array([gamma(ui) for ui in u])
    else:
        if u <= 0.0031308:
            return 12.92 * u
        else:
            return (1.055*u**(1/2.4) - 0.055)
        
def xyY_to_XYZ(xyY):
    x, y, Y = xyY[0], xyY[1], xyY[2]
    X = Y/y * x
    Z = Y/y * (1-x-y)
    return np.array([X, Y, Z])

def xyY_to_sRGB(xyY):
    XYZ = xyY_to_XYZ(xyY)
    RGB = XYZ_to_RGBlinear(XYZ)
    sRGB = gamma(RGB)
    return sRGB

In [3]:
D65_xyY = np.array([0.3127, 0.3290, 1])
print("D65 xyY:\t", D65_xyY)
print("D65 sRGB:\t", xyY_to_sRGB(D65_xyY))
print()
green_xyY = np.array([0.1, 0.6, 1])
print("greenish xyY:\t", green_xyY)
print("greenish sRGB:\t", xyY_to_sRGB(green_xyY))

D65 xyY:	 [0.3127 0.329  1.    ]
D65 sRGB:	 [1. 1. 1.]

greenish xyY:	 [0.1 0.6 1. ]
greenish sRGB:	 [-16.10512759   1.27233912   0.61287396]


## sRGB -> xyY

In [4]:
def RGBlinear_to_XYZ(RGB):
    M = np.array([[0.41239080, 0.35758434, 0.18048079],
                  [0.21263901, 0.71516868, 0.07219232],
                  [0.01933082, 0.11919478, 0.95053215]])
    return M.dot(RGB)
    
def gamma_inv(u):
    if isinstance(u, np.ndarray):
        return np.array([gamma(ui) for ui in u])
    else:
        if u <= 0.04045:
            return u/12.92
        else:
            return (u+0.055)**(2.4)/1.055
        
def XYZ_to_xyY(XYZ):
    X, Y, Z = XYZ[0], XYZ[1], XYZ[2]
    XYZ_sum = np.sum(XYZ)
    x = X / XYZ_sum
    y = Y / XYZ_sum
    # z = 1 - x - y
    return np.array([x, y, Y])

def sRGB_to_xyY(sRGB):
    RGB = gamma_inv(sRGB)
    XYZ = RGBlinear_to_XYZ(RGB)
    xyY = XYZ_to_xyY(XYZ)
    return xyY

In [6]:
D65_sRGB = np.array([1,1,1])
print("White Point sRGB:\t", D65_sRGB)
print("White Point xyY:\t", sRGB_to_xyY(D65_sRGB))
green_sRGB = np.array([0,1,0])
print("Green sRGB:\t", green_sRGB)
print("Green xyY:\t", sRGB_to_xyY(green_sRGB))

White Point sRGB:	 [1 1 1]
White Point xyY:	 [0.3127     0.329      1.00000001]
Green sRGB:	 [0 1 0]
Green xyY:	 [0.3        0.6        0.71516868]
